In [1]:
import os

os.chdir("/home/hack12/ensemble-ai2024/")

In [2]:
import sys

sys.path.append("..")

In [3]:
import random
from pathlib import Path
from taskdataset import TaskDataset
import hydra
import numpy as np
import torch
import wandb
from omegaconf import omegaconf, OmegaConf
from torch import optim, nn
from torch.utils.data import DataLoader
from torchvision.transforms import RandomAffine
from tqdm import tqdm
import random
from src.config import Config

# from src.data.custom_dataset import EncodingsToLabels, EncodingsDataset
from src.models.linear_head import Net, MapperNet
from src.transforms.affine import (
    AffineTransform,
    AffineAndPadAndShuffleTransform,
    PadAndShuffleTransform,
)
from src.transforms.binary import BinaryTransform
import json
from typing import List
import requests
import argparse
from torch.utils.data import Dataset

# import end2end_stealing.vision_transformer as vits
import torchvision.models as models
import torchvision
import numpy as np

In [8]:
def train_binary(epoch, model, train_loader, criterion, optimizer, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        # output = nn.Sigmoid()(output)
        # pred = (output > 0.5).float()
        target = target.to(device)

        # print('pred', pred)
        # print('target', target)
        loss = criterion(output, target.to(device))

        # print(loss)
        loss.backward()
        optimizer.step()
    print(
        "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epoch,
            batch_idx * len(data),
            len(train_loader.dataset),
            100.0 * batch_idx / len(train_loader),
            loss.item(),
        )
    )

In [9]:
class EncodingsDataset(Dataset):
    def __init__(self, transformed_encodings, encodings):
        self.transformed_encodings = torch.from_numpy(
            transformed_encodings.astype("float32")
        )
        self.encodings = torch.from_numpy(encodings.astype("float32"))

    def __len__(self):
        return len(self.transformed_encodings)

    def __getitem__(self, idx):
        return self.transformed_encodings[idx], self.encodings[idx]

In [10]:
class MapperNet(nn.Module):
    def __init__(self, n_inputs, n_outputs, n_hidden):
        super().__init__()
        self.fc1 = nn.Linear(n_inputs, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_outputs)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [11]:
SEED = 0
torch.manual_seed(SEED)
# torch.use_deterministic_algorithms(True)
torch.backends.cudnn.benchmark = False
random.seed(SEED)
np.random.seed(SEED)

device = torch.device(f"cuda") if torch.cuda.is_available() else torch.device("cpu")

In [12]:
dataset = torch.load(
    "/home/bartosz.cywinski@aiclearing.com/code/ensemble-ai2024/end2end-stealing/checkpoints/SybilAttack.pt"
)
dataset_ids = dataset.ids

In [32]:
learning_rate = 3e-5
batch_size = 64
mapper_epochs = 2500

In [14]:
overlap = 1000

In [15]:
results = {}

In [17]:
dataset_ids = dataset.ids

In [19]:
directory_path = (
    "/home/bartosz.cywinski@aiclearing.com/code/ensemble-ai2024/sybil_affine"
)

In [21]:
import os

In [22]:
files = os.listdir(directory_path)

# Filter files that start with 'B'
filtered_files = [file for file in files if file.startswith("B")]

In [23]:
A_data = np.load(
    "/home/bartosz.cywinski@aiclearing.com/code/ensemble-ai2024/sybil_affine/A.npy"
)

In [25]:
B_data_queries = [
    np.load(directory_path + "/" + file_name) for file_name in filtered_files
]

In [26]:
B_data_train = [B_data[:overlap] for B_data in B_data_queries]

In [27]:
B_data_test = [B_data[overlap:] for B_data in B_data_queries]

In [28]:
A_data_train = A_data[:overlap]

In [29]:
results["ids"] = dataset_ids
results["features"] = A_data

In [30]:
MODEL_OUT_IN_SIZE = A_data_train.shape[1]
HIDDEN = 2 * 384

In [33]:
for B_data_train, B_data_test in zip(B_data_train, B_data_test):
    train_dataset = EncodingsDataset(B_data_train, A_data_train)

    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=batch_size, shuffle=True
    )

    mapper = MapperNet(MODEL_OUT_IN_SIZE, MODEL_OUT_IN_SIZE, n_hidden=HIDDEN).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(mapper.parameters(), lr=learning_rate)

    for epoch in range(1, mapper_epochs):
        train_binary(epoch, mapper, train_loader, criterion, optimizer, device=device)

    reconstructed_encodings_test = []
    with torch.no_grad():
        for encoding in tqdm(B_data_test):
            encoding = torch.Tensor(encoding)
            reconstructed_encoding = mapper(encoding.to(device))
            reconstructed_encoding = nn.Sigmoid()(reconstructed_encoding)
            reconstructed_encoding_pred = (reconstructed_encoding > 0.5).float()
            reconstructed_encodings_test.append(
                reconstructed_encoding_pred.detach().to("cpu")
            )
        reconstructed_encodings_test = torch.cat(reconstructed_encodings_test).reshape(
            len(B_data_test), -1
        )

    results["features"] = np.concatenate(
        (results["features"], reconstructed_encodings_test.numpy())
    )

Train Epoch: 1 [248/1000 (97%)]	Loss: 0.748437
Train Epoch: 2 [248/1000 (97%)]	Loss: 0.536399
Train Epoch: 3 [248/1000 (97%)]	Loss: 0.717051
Train Epoch: 4 [248/1000 (97%)]	Loss: 0.613086
Train Epoch: 5 [248/1000 (97%)]	Loss: 0.521533
Train Epoch: 6 [248/1000 (97%)]	Loss: 0.589056
Train Epoch: 7 [248/1000 (97%)]	Loss: 0.561741
Train Epoch: 8 [248/1000 (97%)]	Loss: 0.507855
Train Epoch: 9 [248/1000 (97%)]	Loss: 0.503945
Train Epoch: 10 [248/1000 (97%)]	Loss: 0.705297
Train Epoch: 11 [248/1000 (97%)]	Loss: 0.487388
Train Epoch: 12 [248/1000 (97%)]	Loss: 0.590091
Train Epoch: 13 [248/1000 (97%)]	Loss: 0.464330
Train Epoch: 14 [248/1000 (97%)]	Loss: 0.528143
Train Epoch: 15 [248/1000 (97%)]	Loss: 0.604429
Train Epoch: 16 [248/1000 (97%)]	Loss: 0.462027
Train Epoch: 17 [248/1000 (97%)]	Loss: 0.611283
Train Epoch: 18 [248/1000 (97%)]	Loss: 0.572730
Train Epoch: 19 [248/1000 (97%)]	Loss: 0.590099
Train Epoch: 20 [248/1000 (97%)]	Loss: 0.590036
Train Epoch: 21 [248/1000 (97%)]	Loss: 0.564073
T

100%|██████████| 1000/1000 [00:00<00:00, 5255.14it/s]


Train Epoch: 1 [248/1000 (97%)]	Loss: 0.591497
Train Epoch: 2 [248/1000 (97%)]	Loss: 0.514058
Train Epoch: 3 [248/1000 (97%)]	Loss: 0.485178
Train Epoch: 4 [248/1000 (97%)]	Loss: 0.506673
Train Epoch: 5 [248/1000 (97%)]	Loss: 0.552398
Train Epoch: 6 [248/1000 (97%)]	Loss: 0.480622
Train Epoch: 7 [248/1000 (97%)]	Loss: 0.611610
Train Epoch: 8 [248/1000 (97%)]	Loss: 0.526283
Train Epoch: 9 [248/1000 (97%)]	Loss: 0.670411
Train Epoch: 10 [248/1000 (97%)]	Loss: 0.685460
Train Epoch: 11 [248/1000 (97%)]	Loss: 0.441081
Train Epoch: 12 [248/1000 (97%)]	Loss: 0.402257
Train Epoch: 13 [248/1000 (97%)]	Loss: 0.742029
Train Epoch: 14 [248/1000 (97%)]	Loss: 0.570701
Train Epoch: 15 [248/1000 (97%)]	Loss: 0.460256
Train Epoch: 16 [248/1000 (97%)]	Loss: 0.686950
Train Epoch: 17 [248/1000 (97%)]	Loss: 0.697453
Train Epoch: 18 [248/1000 (97%)]	Loss: 0.710881
Train Epoch: 19 [248/1000 (97%)]	Loss: 0.682810
Train Epoch: 20 [248/1000 (97%)]	Loss: 0.480754
Train Epoch: 21 [248/1000 (97%)]	Loss: 0.817093
T

100%|██████████| 1000/1000 [00:00<00:00, 4867.84it/s]


Train Epoch: 1 [248/1000 (97%)]	Loss: 0.736023
Train Epoch: 2 [248/1000 (97%)]	Loss: 0.623215
Train Epoch: 3 [248/1000 (97%)]	Loss: 0.575141
Train Epoch: 4 [248/1000 (97%)]	Loss: 0.714506
Train Epoch: 5 [248/1000 (97%)]	Loss: 0.489644
Train Epoch: 6 [248/1000 (97%)]	Loss: 0.494917
Train Epoch: 7 [248/1000 (97%)]	Loss: 0.653178
Train Epoch: 8 [248/1000 (97%)]	Loss: 0.590046
Train Epoch: 9 [248/1000 (97%)]	Loss: 0.649482
Train Epoch: 10 [248/1000 (97%)]	Loss: 0.629387
Train Epoch: 11 [248/1000 (97%)]	Loss: 0.852800
Train Epoch: 12 [248/1000 (97%)]	Loss: 0.700486
Train Epoch: 13 [248/1000 (97%)]	Loss: 0.502700
Train Epoch: 14 [248/1000 (97%)]	Loss: 0.524053
Train Epoch: 15 [248/1000 (97%)]	Loss: 0.771072
Train Epoch: 16 [248/1000 (97%)]	Loss: 0.717789
Train Epoch: 17 [248/1000 (97%)]	Loss: 0.648552
Train Epoch: 18 [248/1000 (97%)]	Loss: 0.600737
Train Epoch: 19 [248/1000 (97%)]	Loss: 0.582328
Train Epoch: 20 [248/1000 (97%)]	Loss: 0.471549
Train Epoch: 21 [248/1000 (97%)]	Loss: 0.593820
T

...

100%|██████████| 1000/1000 [00:02<00:00, 367.80it/s]


Train Epoch: 1 [248/1000 (97%)]	Loss: 0.630895
Train Epoch: 2 [248/1000 (97%)]	Loss: 0.658049
Train Epoch: 3 [248/1000 (97%)]	Loss: 0.544134
Train Epoch: 4 [248/1000 (97%)]	Loss: 0.488017
Train Epoch: 5 [248/1000 (97%)]	Loss: 0.550538
Train Epoch: 6 [248/1000 (97%)]	Loss: 0.418921
Train Epoch: 7 [248/1000 (97%)]	Loss: 0.598646
Train Epoch: 8 [248/1000 (97%)]	Loss: 0.557219
Train Epoch: 9 [248/1000 (97%)]	Loss: 0.527961
Train Epoch: 10 [248/1000 (97%)]	Loss: 0.583926
Train Epoch: 11 [248/1000 (97%)]	Loss: 0.608529
Train Epoch: 12 [248/1000 (97%)]	Loss: 0.491064
Train Epoch: 13 [248/1000 (97%)]	Loss: 0.623612
Train Epoch: 14 [248/1000 (97%)]	Loss: 0.666800
Train Epoch: 15 [248/1000 (97%)]	Loss: 0.461438
Train Epoch: 16 [248/1000 (97%)]	Loss: 0.430052
Train Epoch: 17 [248/1000 (97%)]	Loss: 0.547155
Train Epoch: 18 [248/1000 (97%)]	Loss: 0.432257
Train Epoch: 19 [248/1000 (97%)]	Loss: 0.408758
Train Epoch: 20 [248/1000 (97%)]	Loss: 0.630790
Train Epoch: 21 [248/1000 (97%)]	Loss: 0.620431
T

100%|██████████| 1000/1000 [00:00<00:00, 2433.48it/s]


Train Epoch: 1 [248/1000 (97%)]	Loss: 0.704666
Train Epoch: 2 [248/1000 (97%)]	Loss: 0.608995
Train Epoch: 3 [248/1000 (97%)]	Loss: 0.672892
Train Epoch: 4 [248/1000 (97%)]	Loss: 0.652507
Train Epoch: 5 [248/1000 (97%)]	Loss: 0.666358
Train Epoch: 6 [248/1000 (97%)]	Loss: 0.522042
Train Epoch: 7 [248/1000 (97%)]	Loss: 0.390449
Train Epoch: 8 [248/1000 (97%)]	Loss: 0.658140
Train Epoch: 9 [248/1000 (97%)]	Loss: 0.607106
Train Epoch: 10 [248/1000 (97%)]	Loss: 0.579918
Train Epoch: 11 [248/1000 (97%)]	Loss: 0.439188
Train Epoch: 12 [248/1000 (97%)]	Loss: 0.488525
Train Epoch: 13 [248/1000 (97%)]	Loss: 0.674069
Train Epoch: 14 [248/1000 (97%)]	Loss: 0.715774
Train Epoch: 15 [248/1000 (97%)]	Loss: 0.629945
Train Epoch: 16 [248/1000 (97%)]	Loss: 0.678081
Train Epoch: 17 [248/1000 (97%)]	Loss: 0.423350
Train Epoch: 18 [248/1000 (97%)]	Loss: 0.449842
Train Epoch: 19 [248/1000 (97%)]	Loss: 0.738470
Train Epoch: 20 [248/1000 (97%)]	Loss: 0.634975
Train Epoch: 21 [248/1000 (97%)]	Loss: 0.622509
T

100%|██████████| 1000/1000 [00:02<00:00, 498.38it/s]


In [34]:
len(results["features"])

20000

In [35]:
np.savez(
    "example_submission_afajniak.npz",
    ids=results["ids"],
    representations=results["features"],
)

In [36]:
def sybil_submit(binary_or_affine: str, path_to_npz_file: str):
    TEAM_TOKEN = "8J40ASDQOjfeeSKL"
    SERVER_URL = "http://34.71.138.79:9090"

    if binary_or_affine not in ["binary", "affine"]:
        raise Exception("Invalid endpoint")

    endpoint = f"/sybil/{binary_or_affine}/submit"
    url = SERVER_URL + endpoint

    with open(path_to_npz_file, "rb") as f:
        response = requests.post(url, files={"file": f}, headers={"token": TEAM_TOKEN})

    if response.status_code == 200:
        print("OK")
        print(response.json())
    else:
        print(
            f"Request submit failed. Status code: {response.status_code}, content: {response.json()}"
        )

In [37]:
sybil_submit(
    "affine",
    "/home/bartosz.cywinski@aiclearing.com/code/ensemble-ai2024/src/example_submission_afajniak.npz",
)

OK
{'score': 26.573927318776168}
